In [164]:
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [856]:
nS = 20
nA = 4
alpha=0.25
N=100
dz = (50 + 50)/(50)
z = np.arange(51) * dz - 50
p = np.random.rand(nS, nA, 51)
sums = p.sum(axis = -1)
p = p/sums[:, :, np.newaxis]
count = np.random.randint(low=1, high=100, size=(nS, nA))

In [857]:
def CVaRoptNorm(x, p, count, z, alpha, c=0.1, N=20, bonus=None):
    # x is array of size [n_samples, 1]
    batchsize = x.shape[0]
    
    Q = np.zeros((batchsize, nA))
    for s in range(batchsize):
        for a in range(nA):
            # Apply Optimism
            if count is not None:
                cdf = np.cumsum(p[x[s], a, :]) - c/np.sqrt(count[x[s], a])
            cdf = np.clip(cdf, a_min=0, a_max=None)
            cdf[-1] = 1 #Match the last one to 1
            # Compute CVaR
            cdf = np.tile(cdf, N).reshape(N, 51)
            tau = np.random.uniform(0, alpha, N).reshape(N, 1)
            idx = np.argmax((cdf > tau) * 1.0, axis=1)
            # Average
            values = z[idx]
            Q[s, a] = np.mean(values)
    return Q

In [869]:
def CVaRoptA(x, p, count, z, alpha, c=0.1, N=20, bonus=None):
    # x is array of size [n_samples, 1]
    batchsize = x.shape[0]
    
    Q = np.zeros((batchsize, nA))
    for a in range(nA):
        # Apply Optimism
        if count is not None:
            cdf = np.cumsum(p[x, a, :], axis=-1) - np.expand_dims(c/np.sqrt(count[x, a]), axis=-1)  
        else:
            cdf = np.cumsum(p[x, a, :], axis=-1) - bonus
        cdf = np.clip(cdf, a_min=0, a_max=None)
        cdf[:, -1] = 1 #Match the last one to 1
        # Compute CVaR
        tau = np.expand_dims(np.random.uniform(0, alpha, N).reshape(N, 1), axis = 0)
        #cdf = np.tile(cdf, N).reshape(batchsize, N, 51)
        cdf = np.expand_dims(cdf, axis = 1)
        idx = np.argmax((cdf > tau) * 1.0, axis=-1)
        # Average
        values = z[idx]
        Q[:, a] = np.mean(values, axis=-1)
    return Q

In [870]:
def CVaRopt(x, p, count, z, alpha, c=0.1, N=20, bonus=None):
    # x is array of size [n_samples, 1]
    batchsize = x.shape[0]
    
    Q = np.zeros((batchsize, nA))
    if count is not None:
        cdf = np.cumsum(p[x, :, :], axis=-1) - np.expand_dims(c/np.sqrt(count[x, :]), axis=-1)
    
    
    cdf = np.clip(np.squeeze(cdf), a_min=0, a_max=1)
    cdf[:, :, -1] = 1 #Match the last one to 1
    tau = np.random.uniform(0, alpha, N)
    cdf = np.expand_dims(cdf, axis = -1)
    idx = np.argmax((cdf > tau[np.newaxis,np.newaxis,np.newaxis,:]) * 1.0, axis=-2)

    values = z[idx]
    Q[:, :] = np.mean(values, axis=-1)
    
#     for a in range(nA):
#         # Apply Optimism
#         #print(np.cumsum(p[x, a, :], axis=-1).shape)
#         #print((c/np.sqrt(count[x, a])).shape)
#         if count is not None:
#             cdf = np.cumsum(p[x, a, :], axis=-1) - np.expand_dims(c/np.sqrt(count[x, a]), axis=-1)
        
        
        
#         # Compute CVaR
#         tau = np.expand_dims(np.random.uniform(0, alpha, N).reshape(N, 1), axis = 0)
#         #cdf = np.tile(cdf, N).reshape(batchsize, N, 51)
#         cdf = np.expand_dims(cdf, axis = 1)
#         idx = np.argmax((cdf > tau) * 1.0, axis=-1)
#         # Average

    return Q

In [871]:
x = np.array([0, 2, 4, 6, 8]*1000)
start = time.time()
print(CVaRopt(x, p, count, z, alpha, c=2, N=N))
end = time.time()
print(end - start)

[[ 44.32  -9.06 -15.38 -19.22]
 [ 13.5   -1.38  -7.16   3.22]
 [-14.5  -10.98  -1.68 -17.92]
 ...
 [-14.5  -10.98  -1.68 -17.92]
 [ 41.78  50.   -19.28  -9.12]
 [ -8.38  -3.88 -12.6   -8.48]]
1.1451525688171387


In [872]:
start = time.time()
print(CVaRoptA(x, p, count, z, alpha, c=20, N=N)) # best
end = time.time()
print(end - start)

[[50. 50. 50. 50.]
 [50. 50. 50. 50.]
 [50. 50. 50. 50.]
 ...
 [50. 50. 50. 50.]
 [50. 50. 50. 50.]
 [50. 50. 50. 50.]]
0.6737716197967529


In [ ]:
start = time.time()
print(CVaRoptNorm(x, p, count, z, alpha, c=20, N=N))
end = time.time()
print(end - start)

In [807]:
def observe_old(x, a, r, nx, terminal, lr, bonus, p, z, ifCVaR=True):
    dz = 2
    # Choose the optimal action a*
    if not ifCVaR: #Normal
        Q_nx = np.sum(p[nx, :, :] * z, axis=1)
        a_star = np.argmax(Q_nx)
    else: # take the argmax of CVaR
        Q_nx = CVaRoptA(x, p, None, z, 0.25, N=N, bonus=bonus)
        a_star = np.argmax(Q_nx)

    m = np.zeros(51) #target distribution
 
    if not terminal:
        # Apply Optimism:
        cdf = np.cumsum(p[nx, a_star, :]) - bonus
        cdf = np.clip(cdf, a_min=0, a_max=1) # Set less than 0 to 0
        cdf[-1] = 1 #set the last to be equal to 1
        cdf[1:] -= cdf[:-1]
        optimistic_pdf = cdf # Set the optimisitc pdf
 
        # Distribute the probability mass
        tz = np.clip(r + 0.95*z,\
                     -50, 50)
        b = (tz - -50)/dz
        l = np.floor(b).astype(np.int32); u = np.ceil(b).astype(np.int32)

        idx = np.arange(51)
        m[l] += optimistic_pdf[idx] * (u-b)

        m[u] += optimistic_pdf[idx] * (b-l)
             # taking into account when l == u
             # will be zero for l<b and 1 for l==b
        #print(np.floor((1 + (l-b))))
        #m[idx] += optimistic_pdf[idx] * np.floor((1 + (l-b)))
        # Terminal State:
    else:
        print('here')
        tz = np.clip(r, -50, 50)
        b = (tz - -50)/dz
        l = int(np.floor(b)); u = int(np.ceil(b))
        if l == u:
            m[l] = 1
        else:
            m[l] = (u-b)
            m[u] = (b-l)
    return m
 #         # Learning with learning rate
 #         self.p[x, a, :] = self.p[x, a, :] + lr * (m - self.p[x, a, :])
 #         # Map back to a probability distribtuion, sum = 1
 #         self.p[x, a, :] /= np.sum(self.p[x, a, :])

In [840]:
def observe_mid(x, a, r, nx, terminal, lr, bonus, p, z, ifCVaR=True):
    dz = 2
    # Choose the optimal action a*
    batchsize = x.shape[0]
    if not ifCVaR: #Normal
        Q_nx = np.sum(p[nx, :, :] * z, axis=-1)
        a_star = np.argmax(Q_nx, axis=-1)
    else: # take the argmax of CVaR
        Q_nx = CVaRoptA(x, p, None, z, 0.25, N=N, bonus=bonus)
        a_star = np.argmax(Q_nx, axis=-1)
    

    m = np.zeros((batchsize, 51)) #target distribution
    for bb in range(batchsize):
        if not terminal[bb]:
            # Apply Optimism:
            cdf = np.cumsum(p[nx[bb], a_star[bb], :]) - bonus
            cdf = np.clip(cdf, a_min=0, a_max=1) # Set less than 0 to 0
            cdf[-1] = 1 #set the last to be equal to 1
            cdf[1:] -= cdf[:-1]
            optimistic_pdf = cdf # Set the optimisitc pdf

            # Distribute the probability mass
            tz = np.clip(r[bb] + 0.95*z,\
                         -50, 50)
            b = (tz - -50)/dz
            l = np.floor(b).astype(np.int32); u = np.ceil(b).astype(np.int32)

            idx = np.arange(51)
            m[bb, l] += optimistic_pdf[idx] * (u-b)

            m[bb, u] += optimistic_pdf[idx] * (b-l)
                 # taking into account when l == u
                 # will be zero for l<b and 1 for l==b
            #print(np.floor((1 + (l-b))))
            #m[idx] += optimistic_pdf[idx] * np.floor((1 + (l-b)))
            # Terminal State:
        else:
            print('here')
            tz = np.clip(r, -50, 50)
            b = (tz - -50)/dz
            l = int(np.floor(b)); u = int(np.ceil(b))
            if l == u:
                m[bb, l] = 1
            else:
                m[bb, l] = (u-b)
                m[bb, u] = (b-l)
    return np.mean(m, axis=0)
 #         # Learning with learning rate
 #         self.p[x, a, :] = self.p[x, a, :] + lr * (m - self.p[x, a, :])
 #         # Map back to a probability distribtuion, sum = 1
 #         self.p[x, a, :] /= np.sum(self.p[x, a, :])

In [851]:
def observe(x, a, r, nx, terminal, lr, bonus, p, z, ifCVaR=True):
    #dz = 2
    # Choose the optimal action a*
    batchsize = x.shape[0]
    if not ifCVaR: #Normal
        Q_nx = np.sum(p[nx, :, :] * z, axis=-1)
        a_star = np.argmax(Q_nx, axis=-1)
    else: # take the argmax of CVaR
        Q_nx = CVaRoptA(x, p, None, z, 0.25, N=N, bonus=bonus)
        a_star = np.argmax(Q_nx, axis=-1)
    
    m = np.zeros((batchsize, 51)).flatten() #target distribution
    #m = np.random.rand(batchsize, 51)
    cdf = np.cumsum(p[nx, a_star, :], axis=-1) - bonus
    cdf = np.clip(cdf, a_min=0, a_max=1) # Set less than 0 to 0
    cdf[:, -1] = 1 #set the last to be equal to 1
    cdf[:, 1:] -= cdf[:, :-1]
    optimistic_pdf = cdf # Set the optimisitc pdf
    #print(optimistic_pdf.shape)
    # Distribute the probability mass
    T = terminal[:, np.newaxis].astype(np.int32)
    
    tz = np.clip(r[:, np.newaxis] + 0.95*z[np.newaxis, :] * (1-T),\
                 -50, 50)
    b = (tz - -50)/dz
    l = np.floor(b).astype(np.int32); u = np.ceil(b).astype(np.int32)
    idx = np.arange(51)
    #print(np.bincount(l.flatten()))
    bbbbb = np.arange(batchsize)[:, np.newaxis]
    l_idx = (51 * bbbbb + l).flatten()
    
    #print(l_idx)
    m[l_idx] += (((1-T) * optimistic_pdf[:, idx] +\
            T) * (u-b)).flatten()
    
    u_idx =  (51 * np.arange(batchsize)[:, np.newaxis] + u).flatten()
    m[u_idx] += (((1-T) * optimistic_pdf[:, idx] +\
            T) * (b-l)).flatten()
    m = m.reshape(batchsize, 51)
    # taking into account when l == u
    # will be zero for l<b and 1 for l==b
    #m[np.tile(idx, batchsize).flatten()] += (((1-terminal)[:, np.newaxis] * optimistic_pdf[:, idx] +\
    #        terminal.astype(np.int32)[:, np.newaxis]) * np.floor((1 + (l-b)))).flatten()
    m = np.mean(m ,axis = 0)
    return m

In [852]:
sss = 10000
x = np.array([0, 2, 4, 6, 8]*sss); a=np.array([0, 1, 2, 1, 3]*sss); r=np.array([0.2, 0.2, -1.2, -1.2, -5.2]*sss)
terminal = np.array([False, False, False, False, False]*sss)
#x=np.array([5, 4]); a=np.array([2, 1]); r=np.array([0.2, 0.2]); terminal=np.array([False, False])
start = time.time()
mm = observe(x, a, r, x+1, terminal, 0.001, 0.1, p, z)
end = time.time()
print(end-start)

start = time.time()
mmm = observe_mid(x, a, r, x+1, terminal, 0.001, 0.1, p, z)
end = time.time()
print(end-start)

m = np.zeros((5*sss, 51))
start = time.time()
for i in range(5*sss):
    m[i, :]= observe_old(x[i, np.newaxis], a[i, np.newaxis], r[i, np.newaxis],\
                x[i, np.newaxis]+1, terminal[i, np.newaxis], 0.001, 0.1, p, z)
end = time.time()
print(end-start)
#print(np.mean(m, axis=0))
print(np.linalg.norm(mm - np.mean(m, axis=0), 2))
print(np.linalg.norm(mmm - np.mean(m, axis=0), 2))

7.891872882843018
10.825649976730347
25.832557916641235
0.015285833235971907
0.012301073612616687


In [782]:
np.bincount([0, 1, 1, 2, 3])

array([1, 2, 1, 1])

In [741]:
a = np.random.rand(2, 3)

In [855]:
25.8/10.8

2.388888888888889

In [853]:
25.8/7.8

3.307692307692308

In [ ]:
        
     '''
         m = np.zeros(self.config.nAtoms) #target distribution
 
         if not terminal:
             # Apply Optimism:
             cdf = np.cumsum(self.p[nx, a_star, :]) - bonus
             cdf = np.clip(cdf, a_min=0, a_max=1) # Set less than 0 to 0
             cdf[-1] = 1 #set the last to be equal to 1
             cdf[1:] -= cdf[:-1]
             optimistic_pdf = cdf # Set the optimisitc pdf
 
             # Distribute the probability mass
             tz = np.clip(r + self.config.gamma * self.z,\
                     self.config.Vmin, self.config.Vmax)
             b = (tz - self.config.Vmin)/self.dz
             l = np.floor(b).astype(np.int32); u = np.ceil(b).astype(np.int32)
             idx = np.arange(self.config.nAtoms)
 
             m[l] += self.p[nx, a_star, idx] * (u-b)
             m[u] += self.p[nx, a_star, idx] * (b-l)

    # Distribute the probability mass
             tz = np.clip(r + self.config.gamma * self.z,\
                     self.config.Vmin, self.config.Vmax)
             b = (tz - self.config.Vmin)/self.dz
             l = np.floor(b).astype(np.int32); u = np.ceil(b).astype(np.int32)
             idx = np.arange(self.config.nAtoms)
 
             m[l] += self.p[nx, a_star, idx] * (u-b)
             m[u] += self.p[nx, a_star, idx] * (b-l)
 
             # taking into account when l == u
             # will be zero for l<b and 1 for l==b
             m[idx] += self.p[nx, a_star, idx] * np.floor((1 + (l-b)))
         # Terminal State:
         else:
             tz = np.clip(r, self.config.Vmin, self.config.Vmax)
             b = (tz - self.config.Vmin)/self.dz
             l = int(np.floor(b)); u = int(np.ceil(b))
             if l == u:
                 m[l] = 1
             else:
                 m[l] = (u-b)
                 m[u] = (b-l)
 
         # Learning with learning rate
         self.p[x, a, :] = self.p[x, a, :] + lr * (m - self.p[x, a, :])
         # Map back to a probability distribtuion, sum = 1
         self.p[x, a, :] /= np.sum(self.p[x, a, :])
    '''